In [1]:
class counter:
    GLOBAL_REF_TRACKER = {}
    def __init__(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] = 1
    def add_ref(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] += 1
    def release_ref(self):
        obj_id = id(self)
        counter.GLOBAL_REF_TRACKER[obj_id] -= 1
        if counter.GLOBAL_REF_TRACKER[obj_id] == 0 :
            print(f"{obj_id} : 객체 소멸")
            del counter.GLOBAL_REF_TRACKER[obj_id]
    def copy(self):
        return counter()

In [4]:
class MemoryObject:
    # 클래스 변수: 전체 객체 관리
    MEM_TRACKER = {}

    def __init__(self, name):
        self.name = name  # 객체 이름
        obj_id = id(self)
        # 객체 등록
        MemoryObject.MEM_TRACKER[obj_id] = {
            "count": 1,
            "children": [],
            "name": self.name
        }

    def add_ref(self):
        obj_id = id(self)
        MemoryObject.MEM_TRACKER[obj_id]["count"] += 1  # 참조 수 증가

    def release_ref(self):
        obj_id = id(self)
        MemoryObject.MEM_TRACKER[obj_id]["count"] -= 1  # 참조 수 감소

        # 참조 수가 0이면 소멸
        if MemoryObject.MEM_TRACKER[obj_id]["count"] == 0:
            print(f"{obj_id} : 객체 소멸")
            del MemoryObject.MEM_TRACKER[obj_id]  # MEM_TRACKER에서 삭제

    def add_child(self, child):
        obj_id = id(self)
        child_id = id(child)
        # 자식 객체 추가 및 참조 수 증가
        MemoryObject.MEM_TRACKER[obj_id]["children"].append(child)
        child.add_ref()
        
    def remove_child(self,clild):
        obj_id = id(self)
        child_id = id(child)
        # 자식 객체가 children 목록에 있는지 확인
        if child in MemoryObject.MEM_TRACKER[obj_id]["children"]:
            # 참조 수 감소 및 자식 제거
            MemoryObject.MEM_TRACKER[obj_id]["children"].remove(child)
            child.release_ref()
        else:
            # 자식이 없을 때 처리 (선택 사항)
            print(f"{child_id} : 자식 객체가 존재하지 않아 삭제할 수 없습니다.")
    
    @classmethod
    def show_tracker(cls):
        # 현재 MEM_TRACKER 상태 출력
        for obj_id, info in cls.MEM_TRACKER.items():
            print(f"ID: {obj_id}, Info: {info}")
    
if __name__ == "__main__":
    print("\n--- [ 순환 참조 생성 및 기본 레퍼런스 카운트 관리 ] ---")
    # 두 객체 생성
    objA = MemoryObject("A")
    objB = MemoryObject("B")

    # 서로를 자식으로 추가하여 순환 참조 형성
    objA.add_child(objB)  # A -> B (B의 카운트 +1)
    objB.add_child(objA)  # B -> A (A의 카운트 +1)

    # 현재 MEM_TRACKER 상태 출력
    print("\nMEM_TRACKER 상태 (순환 참조 형성 후):")
    for obj_id, info in MemoryObject.MEM_TRACKER.items():
        print(f" id: {obj_id}, name: {info['name']}, count: {info['count']}")

    # 외부(root) 참조 제거: 여기서는 부모 자체를 None 처리
    # (하지만 내부 순환 참조 때문에 카운트는 2로 남음)
    objA.release_ref()  # 외부 참조 제거: A의 count 감소
    objB.release_ref()  # B의 count 감소 (원래 2 -> 1)

    print("\n외부 참조 제거 후 MEM_TRACKER 상태:")
    for obj_id, info in MemoryObject.MEM_TRACKER.items():
        print(f" id: {obj_id}, name: {info['name']}, count: {info['count']}")
    print("\n※ 단순 release_ref만으로는 서로 참조하는 순환 구조 때문에 count가 0이 되지 않음.")



--- [ 순환 참조 생성 및 기본 레퍼런스 카운트 관리 ] ---

MEM_TRACKER 상태 (순환 참조 형성 후):
 id: 4534667216, name: A, count: 2
 id: 4534664720, name: B, count: 2

외부 참조 제거 후 MEM_TRACKER 상태:
 id: 4534667216, name: A, count: 1
 id: 4534664720, name: B, count: 1

※ 단순 release_ref만으로는 서로 참조하는 순환 구조 때문에 count가 0이 되지 않음.


In [6]:
import copy

# 🔵 고객 기본 클래스
class Customer:
    def __init__(self, name, birthdate, grade, address, purchase_history):
        self._info = {
            "name": name,
            "birthdate": birthdate,
            "grade": grade,
            "address": address,
            "purchase_history": purchase_history  # 리스트로 관리
        }

    def get_info(self):
        # 고객 정보 출력
        return self._info

    def get_snapshot(self):
        # 고객 정보를 얕은 복사로 저장
        return copy.deepcopy(self._info)  # 깊은 복사로 변경 (중첩 데이터 보호)

    def rollback(self, snapshot):
        # 스냅샷을 이용해 정보 복원
        self._info = copy.deepcopy(snapshot)  # 깊은 복사로 변경

    def __str__(self):
        # 객체 출력 시 정보가 보이도록 추가
        return str(self._info)


# 🟢 일반 고객 (가변 객체)
class RegularCustomer(Customer):
    def update_info(self, key, new_value):
        # key가 있으면 값 수정
        if key in self._info:
            self._info[key] = new_value
        else:
            print(f"'{key}' 항목이 존재하지 않습니다.")


# 🟡 VIP 고객 (불변 객체)
class VIPCustomer(Customer):
    def update_info(self, key, new_value):
        # VIP 고객은 정보를 수정할 수 없으므로, 새 객체를 반환
        if key in self._info:
            # 기존 정보를 복사 후 새 객체 반환
            new_info = copy.deepcopy(self._info)
            new_info[key] = new_value
            return VIPCustomer(**new_info)
        else:
            print(f"'{key}' 항목이 존재하지 않습니다.")
            return self  # 변경 불가 시 자기 자신 반환


if __name__ == "__main__":
    print("=== 고객 관리 시스템 기능 테스트 ===")

    # 초기 정보 (중첩 자료 포함 - purchase_history)
    vip_info = {
        "name": "Alice",
        "birthdate": "1990-01-01",
        "grade": "VIP",
        "address": "Seoul",
        "purchase_history": [{"item": "Laptop", "price": 1500}]
    }

    regular_info = {
        "name": "Bob",
        "birthdate": "1985-05-12",
        "grade": "Regular",
        "address": "Busan",
        "purchase_history": [{"item": "Smartphone", "price": 800}]
    }

    # VIP 고객 (불변)
    vip = VIPCustomer(**vip_info)
    print("초기 VIP 고객:", vip)

    # 스냅샷 저장
    vip_snapshot = vip.get_snapshot()

    # Regular 고객 (가변)
    reg = RegularCustomer(**regular_info)
    print("초기 Regular 고객:", reg)

    reg_snapshot = reg.get_snapshot()

    # 업데이트 시도
    # VIP 고객 업데이트: 새 객체 반환
    vip_updated = vip.update_info("address", "Incheon")

    # Regular 고객 업데이트: in-place 변경
    reg.update_info("address", "Daegu")

    # 구매 이력 수정: 중첩 자료 수정 효과 확인
    vip_updated.get_info()["purchase_history"][0]["price"] = 1400
    reg.get_info()["purchase_history"][0]["price"] = 750

    print("\n업데이트 후 VIP 고객:")
    print("원본:", vip)  # 원본은 변화 없음 (불변)
    print("업데이트된 VIP:", vip_updated)
    print("VIP 스냅샷:", vip_snapshot)

    print("\n업데이트 후 Regular 고객:")
    print("현재 상태:", reg)
    print("Regular 스냅샷:", reg_snapshot)

    # 중첩 자료가 mutable이므로, regular 스냅샷(얕은 복사)는 같이 변했을 수 있음.
    # 롤백 테스트
    reg.rollback(reg_snapshot)
    print("\nRegular 고객 롤백 후:", reg)

    # VIP 고객 롤백: 새 객체 반환
    vip_rolled_back = vip_updated.rollback(vip_snapshot)
    print("\nVIP 고객 롤백 후:", vip_rolled_back)


=== 고객 관리 시스템 기능 테스트 ===
초기 VIP 고객: {'name': 'Alice', 'birthdate': '1990-01-01', 'grade': 'VIP', 'address': 'Seoul', 'purchase_history': [{'item': 'Laptop', 'price': 1500}]}
초기 Regular 고객: {'name': 'Bob', 'birthdate': '1985-05-12', 'grade': 'Regular', 'address': 'Busan', 'purchase_history': [{'item': 'Smartphone', 'price': 800}]}

업데이트 후 VIP 고객:
원본: {'name': 'Alice', 'birthdate': '1990-01-01', 'grade': 'VIP', 'address': 'Seoul', 'purchase_history': [{'item': 'Laptop', 'price': 1500}]}
업데이트된 VIP: {'name': 'Alice', 'birthdate': '1990-01-01', 'grade': 'VIP', 'address': 'Incheon', 'purchase_history': [{'item': 'Laptop', 'price': 1400}]}
VIP 스냅샷: {'name': 'Alice', 'birthdate': '1990-01-01', 'grade': 'VIP', 'address': 'Seoul', 'purchase_history': [{'item': 'Laptop', 'price': 1500}]}

업데이트 후 Regular 고객:
현재 상태: {'name': 'Bob', 'birthdate': '1985-05-12', 'grade': 'Regular', 'address': 'Daegu', 'purchase_history': [{'item': 'Smartphone', 'price': 750}]}
Regular 스냅샷: {'name': 'Bob', 'birthdate': 